In [1]:
from datasets import load_dataset
datasets = load_dataset('json', data_files="alpaca-tw.json")
datasets

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 52002
    })
})

### 將資料更新為llama3.1要求的資料格式
```
{"role": "system", "content": "You are an assistant"}
{"role": "user", "content": "What is 2+2?"}
{"role": "assistant", "content": "It's 4."}
```

In [7]:
dataset = datasets['train']
items = []
for item_dict in dataset:
    inner_item = []
    inner_item.append({'role':'system','content':item_dict['instruction']})
    inner_item.append({'role':'user','content':item_dict['input']})
    inner_item.append({'role':'assistant','content':item_dict['output']})
    items.append(inner_item)
print(items[0])


[{'role': 'system', 'content': '給出三個保持健康的提示。'}, {'role': 'user', 'content': ''}, {'role': 'assistant', 'content': '1. 飲食均衡，確保攝取足夠的水果和蔬菜。\n2. 定期運動，保持身體活躍和強壯。\n3. 睡眠充足，保持一致的睡眠時間表。'}]
Dataset({
    features: ['conversations'],
    num_rows: 52002
})

Sample conversation: [{'content': '給出三個保持健康的提示。', 'role': 'system'}, {'content': '', 'role': 'user'}, {'content': '1. 飲食均衡，確保攝取足夠的水果和蔬菜。\n2. 定期運動，保持身體活躍和強壯。\n3. 睡眠充足，保持一致的睡眠時間表。', 'role': 'assistant'}]

Sample conversation: [{'content': '給出三個保持健康的提示。', 'role': 'system'}, {'content': '', 'role': 'user'}, {'content': '1. 飲食均衡，確保攝取足夠的水果和蔬菜。\n2. 定期運動，保持身體活躍和強壯。\n3. 睡眠充足，保持一致的睡眠時間表。', 'role': 'assistant'}]


In [8]:
from datasets import Dataset

# 建立新的dataset with conversations field
new_dataset = Dataset.from_dict({
    'conversations': items
})

print(new_dataset)
print("\nSample conversation:", new_dataset['conversations'][0])


Dataset({
    features: ['conversations'],
    num_rows: 52002
})

Sample conversation: [{'content': '給出三個保持健康的提示。', 'role': 'system'}, {'content': '', 'role': 'user'}, {'content': '1. 飲食均衡，確保攝取足夠的水果和蔬菜。\n2. 定期運動，保持身體活躍和強壯。\n3. 睡眠充足，保持一致的睡眠時間表。', 'role': 'assistant'}]


In [9]:
from datasets import DatasetDict

#轉換為DatasetDict
dataset_dict = DatasetDict({
    'train':new_dataset
})
print(dataset_dict)
print("\nFeatures:", dataset_dict['train'].features)

DatasetDict({
    train: Dataset({
        features: ['conversations'],
        num_rows: 52002
    })
})

Features: {'conversations': [{'content': Value(dtype='string', id=None), 'role': Value(dtype='string', id=None)}]}


### 上傳至hugging_face

In [11]:
from huggingface_hub import HfApi, notebook_login
api = HfApi()
notebook_login() #this is likely needed for authentication
dataset_dict.push_to_hub('roberthsu2003/for_llama3_Instruct',private=False)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/53 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/roberthsu2003/for_llama3_Instruct/commit/defb9ad5c3a2b18d1042a57993b1a0be15e8f8ab', commit_message='Upload dataset', commit_description='', oid='defb9ad5c3a2b18d1042a57993b1a0be15e8f8ab', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/roberthsu2003/for_llama3_Instruct', endpoint='https://huggingface.co', repo_type='dataset', repo_id='roberthsu2003/for_llama3_Instruct'), pr_revision=None, pr_num=None)

## 手動增加一筆資料

In [ ]:
# 建立新的對話資料
new_conversation = [
    {'role': 'system', 'content': '這是新增的系統指令'},
    {'role': 'user', 'content': '這是使用者的輸入'},
    {'role': 'assistant', 'content': '這是助理的回答'}
]

# 將新的對話資料加入現有的 dataset
dataset_dict['train'] = dataset_dict['train'].add_item({
    'conversations': new_conversation
})

# 驗證新增的資料
print("資料集大小:", len(dataset_dict['train']))
print("\n最後一筆資料:", dataset_dict['train'][-1])